# Test if merge_bn does not work with any layer from QuantMultiheadAttention

In [2]:
from brevitas import nn as qnn
from brevitas.nn.utils import merge_bn
import qtransform
import torch

In [5]:
N,C, L = (12, 64, 256)
num_heads = 2
quant_mha = qnn.QuantMultiheadAttention(num_heads=num_heads, embed_dim=L)
bn = torch.nn.BatchNorm1d(C)

In [23]:
layers = list(quant_mha._modules.keys())
layers

['in_proj',
 'out_proj',
 'softmax_input_quant',
 'attn_output_weights_quant',
 'q_scaled_quant',
 'k_transposed_quant',
 'v_quant']

In [ ]:
#merge_bn updates weight and bias parameters


In [29]:
success = list()
for layer in layers:
    quant_mha_layer = getattr(quant_mha, layer)
    print(f'Passing Layer: {layer} ', end="")
    try:
        print(f', weight: {quant_mha_layer.weight.size()}, bias: {quant_mha_layer.bias.size()}', end="")
        merge_bn(quant_mha_layer, bn)
        success.append(layer)
    except:
        pass
    print("")
print(f'merge_bn worked for: {success}')

Passing Layer: in_proj , weight: torch.Size([768, 256]), bias: torch.Size([768])
Passing Layer: out_proj , weight: torch.Size([256, 256]), bias: torch.Size([256])
Passing Layer: softmax_input_quant 
Passing Layer: attn_output_weights_quant 
Passing Layer: q_scaled_quant 
Passing Layer: k_transposed_quant 
Passing Layer: v_quant 
merge_bn worked for: []


odict_keys(['in_proj', 'out_proj', 'softmax_input_quant', 'attn_output_weights_quant', 'q_scaled_quant', 'k_transposed_quant', 'v_quant'])

In [ ]:
#simulate training
prompt = torch.randn(N,C,L)